In [29]:
from sqlite3 import Timestamp
# pip install chromedriver-autoinstaller
import chromedriver_autoinstaller as AutoChrome
import os
import shutil
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
def chromedriver_update():
    chrome_ver = AutoChrome.get_chrome_version().split('.')[0]   # 현재 버전

    current_list = os.listdir(os.getcwd())                 # 현재 경로의 모든 객체들
    chrome_list = []
    for i in current_list:
        path = os.path.join(os.getcwd(), i)                 # 현재 경로의 모든객체의 전체경로
        if os.path.isdir(path):                             # 그 경로가 폴더인지 확인
            if 'chromedriver.exe' in os.listdir(path):      # 폴더면 안에 chromedriver.exe가 있는지 확인
                chrome_list.append(i)                       # 있는경우 chrome_list에 추가

    old_version = list(set(chrome_list)-set([chrome_ver]))  # 그중에 최신버전은 제외

    for i in old_version:
        path = os.path.join(os.getcwd(),i)                  # 구버전이 있는 폴더의 경로 
        shutil.rmtree(path)                                 # 그 경로 삭제

    if not chrome_ver in current_list:                     # 최신버전 폴더가 현재 경로에 없으면
        AutoChrome.install(True)                           # 크롬드라이버 설치
    else:                                                  # 아니면 무시
        pass

In [6]:
chromedriver_update()

In [17]:
#################################
# 최종코드
#################################
chromedriver_update()
chrome_ver = AutoChrome.get_chrome_version().split('.')[0]
driver = webdriver.Chrome(f"./{chrome_ver}/chromedriver")


naver_map_search_url = f"https://m.map.naver.com/search2/interestSpot.naver?type=BANK"
driver.get(naver_map_search_url)

df = {'name' : [], 'add':[], 'call':[], 'dong':[]}

keyword = ['진영읍','주촌면','진례면','한림면','생림면','상동면','대동면','동상동',
            '회현동','부원동','내외동','북부동','칠산서부동','활천동','삼안동','불암동',
            '장유1동', '장유2동', '장유3동']

for dong in keyword:
    driver.find_element_by_xpath('//*[@id="_changeLocation"]').send_keys('\n')
    driver.find_element_by_xpath('/html/body/div[9]/div/div/form/div/div/input').send_keys('김해시' + dong)
    time.sleep(1)
    driver.find_element_by_xpath('/html/body/div[9]/div/div/form/div/button').click()
    time.sleep(1)
    driver.find_element_by_xpath('/html/body/div[9]/div/div/div/div/div/div').click()
    time.sleep(1)

    l = driver.find_elements_by_xpath('//*[@id="ct"]/div[2]/ul/li')
    print(dong,':', len(l),'개')

    for n in range(len(l)):
        df['name'].append(l[n].get_attribute('data-title'))
        df['call'].append(l[n].get_attribute('data-tel'))

        a = driver.find_elements_by_xpath(f'//*[@id="ct"]/div[2]/ul/li[{n+1}]/div[1]/div[1]/div/a')
        a = a[0].text
        df['add'].append(a.split('\n')[1])
        df['dong'].append(dong)


진영읍 : 65 개
주촌면 : 17 개
진례면 : 4 개
한림면 : 1 개
생림면 : 2 개
상동면 : 4 개
대동면 : 4 개
동상동 : 75 개
회현동 : 75 개
부원동 : 75 개
내외동 : 75 개
북부동 : 66 개
칠산서부동 : 61 개
활천동 : 71 개
삼안동 : 75 개
불암동 : 25 개
장유1동 : 75 개
장유2동 : 75 개
장유3동 : 56 개


In [18]:
len(df['name'])

901

In [19]:
df1 = pd.DataFrame(df)

In [20]:
df1

name                                add          call  dong
0     부원새마을금고 진영1지점                경상남도 김해시 진영읍 여래로 22  055-343-9700   진영읍
1       신한은행 진영금융센터          경상남도 김해시 진영읍 장등로 8 (신한빌딩)  055-345-8800   진영읍
2         우리은행 진영지점  경상남도 김해시 진영읍 김해대로361번길 2 진영그랜드프라자  055-345-0811   진영읍
3       NH농협은행 진영지점            경상남도 김해시 진영읍 진영로 182 농협  055-343-2081   진영읍
4           진영농협 본점               경상남도 김해시 진영읍 진영로 185  055-345-7711   진영읍
..              ...                                ...           ...   ...
896  세븐 장유대청푸르지 ATM                 경상남도 김해시 반룡로 63 1층                장유3동
897  GS25 장유번화점 ATM               경상남도 김해시 계동로212번길 18                장유3동
898   CU 율하신도시점 ATM                  경상남도 김해시 율하1로 154                장유3동
899  GS25 율현주공점 ATM            경상남도 김해시 율하2로 175 (율하동)                장유3동
900   세븐 장유율현마을 ATM                   경상남도 김해시 율하로 383                장유3동

[901 rows x 4 columns]

In [21]:
import json
import requests

class KakaoLocalAPI:
    """
    Kakao Local API 컨트롤러
    """

    def __init__(self, rest_api_key):
        """
        Rest API키 초기화 및 기능 별 URL 설정
        """

        # REST API 키 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}

        # 01 주소 검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"

    def search_address(self, query, analyze_type=None, page=None, size=None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}

        if analyze_type != None:
            params["analyze_type"] = f"{analyze_type}"

        if page != None:
            params['page'] = f"{page}"

        if size != None:
            params['size'] = f"{size}"

        res = requests.get(self.URL_01, headers=self.headers, params=params)
        document = json.loads(res.text)

        return document

In [24]:
import api  # 개인 api를 숨기기 위해 만든 모듈

In [25]:
rest_api_key = api.kakao_api  # 이렇게 사용하면 깃허브에 공유하여 사용해도 개인코드를 숨길 수 있음.

kakao = KakaoLocalAPI(rest_api_key)

In [79]:
df1['kakao_add'] = None
drop_li = []
for n in tqdm(range(len(df1))):
    add = df1['add'][n]
    
    if '김해시' not in add:
        drop_li.append(n)
        continue
        
    a = df1['add'][n].split()[2]
    if '읍' in a or '면' in a:
        df1['kakao_add'][n] = a
        continue
    else:
        add = add.split()
        add = ' '.join(add[:4])
        
    if df1['name'][n] == '김해 하랄아시아마트 ATM':
        add = '경상남도 김해시 왕릉길 4'
    elif df1['name'][n] == '롯데시네마-김해부원 ATM':
        add = '경남 김해시 김해대로 2352'
    elif df1['name'][n] == '서김해새마을금고 중앙지점 점두365 ATM':
        add = '경남 김해시 내외로 79'
    elif df1['name'][n] == '하나은행365 한국가스공사 부산/경남지역본부 ATM':
        add = '김해시 안동 540-4'    

    kakao_add = kakao.search_address(query=add)['documents']
    df1['kakao_add'][n] = kakao_add[0]['address']['region_3depth_h_name']


  0%|          | 0/901 [00:00<?, ?it/s]

In [87]:
df1.drop(drop_li, axis=0, inplace=True)

In [88]:
df1.to_csv('./전처리/금융기관.csv',encoding='cp949', index=False)

In [89]:
df1 = pd.read_csv('./전처리/금융기관.csv',encoding='cp949')

In [90]:
df2 = df1
df2

name                                add          call  dong  \
0     부원새마을금고 진영1지점                경상남도 김해시 진영읍 여래로 22  055-343-9700   진영읍   
1       신한은행 진영금융센터          경상남도 김해시 진영읍 장등로 8 (신한빌딩)  055-345-8800   진영읍   
2         우리은행 진영지점  경상남도 김해시 진영읍 김해대로361번길 2 진영그랜드프라자  055-345-0811   진영읍   
3       NH농협은행 진영지점            경상남도 김해시 진영읍 진영로 182 농협  055-343-2081   진영읍   
4           진영농협 본점               경상남도 김해시 진영읍 진영로 185  055-345-7711   진영읍   
..              ...                                ...           ...   ...   
886  세븐 장유대청푸르지 ATM                 경상남도 김해시 반룡로 63 1층           NaN  장유3동   
887  GS25 장유번화점 ATM               경상남도 김해시 계동로212번길 18           NaN  장유3동   
888   CU 율하신도시점 ATM                  경상남도 김해시 율하1로 154           NaN  장유3동   
889  GS25 율현주공점 ATM            경상남도 김해시 율하2로 175 (율하동)           NaN  장유3동   
890   세븐 장유율현마을 ATM                   경상남도 김해시 율하로 383           NaN  장유3동   

    kakao_add  
0         진영읍  
1         진영읍  
2         진영읍  
3         진영읍  
4         진영읍  
..        ...  
886      장유2동  
887      장유2동  
888      장유3동  
889      장유3동  
890      장유3동  

[891 rows x 5 columns]

In [91]:
for a in range(len(df2)):
    if df2['dong'][a] != df2['kakao_add'][a]:
        df2['dong'][a] = df2['kakao_add'][a]

In [92]:
df2.drop('dong', axis=1, inplace=True)

In [95]:
df2.sort_values('name', inplace=True)
df2.reset_index(drop=True, inplace=True)

In [100]:
drop_li = []
for n in tqdm(range(len(df2))):
    if n == 0:
        bf_name = df2['name'][n]
        bf_add = df2['add'][n]
        continue
    af_name = df2['name'][n]
    af_add = df2['add'][n]
    
    if (af_name == bf_name) and (af_add == bf_add):
        drop_li.append(n)
    bf_name, bf_add = af_name, af_add

  0%|          | 0/891 [00:00<?, ?it/s]

In [102]:
df2.drop(drop_li, axis=0, inplace=True)

In [105]:
df2.to_csv('./전처리/금융기관_최종.csv',encoding='cp949', index=False)